In [1]:
import convokit
from convokit import Corpus, PairedPrediction, download, Pairer

In [2]:
corpus = Corpus(filename=download('friends-corpus'))

Dataset already exists at /Users/calebchiam/.convokit/downloads/friends-corpus


In [3]:
corpus.print_summary_stats()

Number of Speakers: 700
Number of Utterances: 67373
Number of Conversations: 3107


In the friends-corpus, Conversations are Scenes, and Utterances correspond to utterances by characters in that scene.

Let's do a basic paired prediction, predicting for whether or not a scene goes on to have all six friends participating, based on features from the first 5 utterances.

## Adding features for first 5 utterances and label for whether all 6 Friends eventually participate

In [4]:
def add_convo_features(convo):
    utts = sorted(list(convo.iter_utterances()), key=lambda utt: utt.id)
    utts_5 = utts[:5]
    speaker_ids = {utt.speaker.id for utt in utts_5}
    main_chars = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Rachel Green', 'Ross Geller']
    
    for char in main_chars:
        convo.add_meta('has_{}_5'.format(char), int(char in speaker_ids))
    
    convo.add_meta('num_main_characters_5', sum(convo.meta['has_{}_5'.format(char)] for char in main_chars))
    
    all_speakers = {utt.speaker.id for utt in utts}
    convo.add_meta('all_friends_present', len(set(main_chars).intersection(all_speakers))==6)

In [5]:
for convo in corpus.iter_conversations():
    add_convo_features(convo)

In [6]:
# Number of Conversations with all friends present
num_convos_with_all_present = 0
for convo in corpus.iter_conversations():
    if convo.meta['all_friends_present'] == 1:
        num_convos_with_all_present += 1
print(num_convos_with_all_present)

311


We will pair based on the episode:

In [7]:
main_chars = ['Monica Geller', 'Joey Tribbiani', 'Chandler Bing', 'Phoebe Buffay', 'Rachel Green', 'Ross Geller']
    
pred_feats = ['has_{}_5'.format(char) for char in main_chars]
pred_feats.append('num_main_characters_5')

In [8]:
pairer = Pairer(obj_type="conversation", pairing_func=lambda convo: str(convo.meta['season']+convo.meta['episode']),
                pos_label_func=lambda convo: convo.meta['all_friends_present'] == 1,
                neg_label_func=lambda convo: convo.meta['all_friends_present'] == 0,
                pair_mode="random"
               )

In [9]:
pp = PairedPrediction(obj_type="conversation", pred_feats=pred_feats)

In [10]:
# Add the pairing information to the conversations

pairer.transform(corpus)

In [11]:
convo = next(corpus.iter_conversations())

In [12]:
# Most conversations will have None in the paired prediction labels

convo.meta

{'season': 's01',
 'episode': 'e01',
 'scene': 'c01',
 'has_Monica Geller_5': 1,
 'has_Joey Tribbiani_5': 1,
 'has_Chandler Bing_5': 1,
 'has_Phoebe Buffay_5': 1,
 'has_Rachel Green_5': 0,
 'has_Ross Geller_5': 0,
 'num_main_characters_5': 4,
 'all_friends_present': True,
 'pair_obj_label': 'pos',
 'pair_id': 's01e01',
 'pair_orientation': 'pos'}

In [13]:
for convo in corpus.iter_conversations():
    if convo.meta['pair_id'] is not None:
        sample_convo = convo
        break

In [14]:
sample_convo.meta

{'season': 's01',
 'episode': 'e01',
 'scene': 'c01',
 'has_Monica Geller_5': 1,
 'has_Joey Tribbiani_5': 1,
 'has_Chandler Bing_5': 1,
 'has_Phoebe Buffay_5': 1,
 'has_Rachel Green_5': 0,
 'has_Ross Geller_5': 0,
 'num_main_characters_5': 4,
 'all_friends_present': True,
 'pair_obj_label': 'pos',
 'pair_id': 's01e01',
 'pair_orientation': 'pos'}

In [15]:
pp.fit(corpus)

Found 149 valid pairs.


In [16]:
pp.summarize(corpus)

Found 149 valid pairs.


0.7449664429530202

In [17]:
pp.get_coefs(feature_names=pred_feats)

,coef
feat_name,
num_main_characters_5,1.048024
has_Monica Geller_5,0.604012
has_Joey Tribbiani_5,0.454098
has_Ross Geller_5,0.422704
has_Phoebe Buffay_5,0.254570
has_Rachel Green_5,0.186515
has_Chandler Bing_5,0.012372
